# SIRS model using SOC approche

In [6]:
import numpy as np
import sys
from matplotlib import pyplot as plt
from matplotlib import colors
from matplotlib import animation
from tempfile import TemporaryFile
import pandas as pd
from tqdm import tqdm

In [7]:
neighbourhood = ((-1,-1), (-1,0), (-1,1), (0,-1), (0, 1), (1,-1), (1,0), (1,1))
S, A, B, AB, a, b, Ab, aB, ab = 0, 1, 2, 3, 4, 5, 6, 7, 8

sicks = []
immune = []

colors_list = ['w','r','y','orange','c','g','b','m','k']
cmap = colors.ListedColormap(colors_list)
bounds = [0,1,2,3,4,5,6,7,8,9]
norm = colors.BoundaryNorm(bounds, cmap.N)

In [8]:
## make list of all lattice index
def make_site_list(length,width):
    s = []
    for i in range(0,length):
        for j in range(0,width):
            s.append([i,j])
            
    return s

In [2]:
### return immune indivisuals to susceptible indivisuals

def health():
    
    for ind in immune:
        if forest[ind[0],ind[1]] == ab:
            d = np.random.choice([a, b], p=(0.5, 0.5))
            if np.random.random() < r:
                forest[ind[0],ind[1]] = d
                    
        elif forest[ind[0],ind[1]] == a or forest[ind[0],ind[1]] == b:
            if np.random.random() < r:
                forest[ind[0],ind[1]] = S
                immune.remove(ind)

In [11]:
### making susceptible indivisual sick with A or B or AB

def lightning(susceptibles):
    
    l = len(susceptibles)
    
    seed = susceptibles[np.random.randint(0,l)]
    forest[seed[0],seed[1]] = A
    sicks.append(seed)
    
    seed = susceptibles[np.random.randint(0,l)]
    forest[seed[0],seed[1]] = B
    sicks.append(seed)
    

### Evolution function

In [12]:
def evolve(sites):
    for sick in sicks:
        ############### AB
        if forest[sick[0],sick[1]] == AB:
            
            if np.random.random() < h:
                c = np.random.choice([Ab,aB],p=(0.5,0.5))
                forest[sick[0],sick[1]] = c
            
            for dx,dy in neighbourhood:
                neig = [(sick[0]+dy)%length,(sick[1]+dx)%width]
                
                if (forest[neig[0],neig[1]] == AB or forest[neig[0],neig[1]] == Ab
                    or forest[neig[0],neig[1]] == aB or forest[neig[0],neig[1]] == ab):
                    
                    continue
                    
                elif forest[neig[0],neig[1]] == A or forest[neig[0],neig[1]] == B:
                    if np.random.random() < m_outbreak:
                        forest[neig[0],neig[1]] = AB
                    continue
                    
                elif forest[neig[0],neig[1]] == a:
                    if np.random.random() < m_outbreak:
                        forest[neig[0],neig[1]] = aB
                        immune.remove([neig[0],neig[1]])
                        sicks.append([neig[0],neig[1]])
                    continue
                        
                elif forest[neig[0],neig[1]] == b:
                    if np.random.random() < m_outbreak:
                        forest[neig[0],neig[1]] = Ab
                        immune.remove([neig[0],neig[1]])
                        sicks.append([neig[0],neig[1]])
                    continue
                
                elif forest[neig[0],neig[1]] == S:
                    if np.random.random() < outbreak:
                        c = np.random.choice([A,B],p=(0.5,0.5))
                        forest[neig[0],neig[1]] = c
                        
                        sicks.append([neig[0],neig[1]])
                    continue
                    
                    
        ############### Ab or A
        if (forest[sick[0],sick[1]] == Ab or forest[sick[0],sick[1]] == A):
            
            if np.random.random() < h:
                sicks.remove([sick[0],sick[1]])
                immune.append([sick[0],sick[1]])
                
                if forest[sick[0],sick[1]] == Ab:
                    forest[sick[0],sick[1]] = ab
                elif forest[sick[0],sick[1]] == A:
                    forest[sick[0],sick[1]] = a
                    
                
            
            for dx,dy in neighbourhood:
                neig = [(sick[0]+dy)%length,(sick[1]+dx)%width]
                
                if (forest[neig[0],neig[1]] == AB or forest[neig[0],neig[1]] == Ab
                    or forest[neig[0],neig[1]] == aB or forest[neig[0],neig[1]] == ab
                    or forest[neig[0],neig[1]] == A or forest[neig[0],neig[1]] == a):
                    continue
                        
                elif forest[neig[0],neig[1]] == B:
                    if np.random.random() < m_outbreak:
                        forest[neig[0],neig[1]] = AB
                    continue
                    
                elif forest[neig[0],neig[1]] == b:
                    if np.random.random() < m_outbreak:
                        forest[neig[0],neig[1]] = Ab
                        immune.remove([neig[0],neig[1]])
                        sicks.append([neig[0],neig[1]])
                    continue
                    
                elif forest[neig[0],neig[1]] == S:
                    if np.random.random() < outbreak:
                        forest[neig[0],neig[1]] = A
                        
                        sicks.append([neig[0],neig[1]]) 
                    continue
    
        ############### aB or B
        if (forest[sick[0],sick[1]] == aB or forest[sick[0],sick[1]] == B):

            if np.random.random() < h:
                if forest[sick[0],sick[1]] == aB:
                    sicks.remove([sick[0],sick[1]])
                    immune.append([sick[0],sick[1]])
                    forest[sick[0],sick[1]] = ab
                
                elif forest[sick[0],sick[1]] == B:
                    sicks.remove([sick[0],sick[1]])
                    immune.append([sick[0],sick[1]])
                    forest[sick[0],sick[1]] = b
                    

            
            for dx,dy in neighbourhood:
                neig = [(sick[0]+dy)%length,(sick[1]+dx)%width]
                
                if (forest[neig[0],neig[1]] == AB or forest[neig[0],neig[1]] == Ab
                    or forest[neig[0],neig[1]] == aB or forest[neig[0],neig[1]] == ab
                    or forest[neig[0],neig[1]] == B or forest[neig[0],neig[1]] == b):
                    continue
                        
                elif forest[neig[0],neig[1]] == A:
                    if np.random.random() < m_outbreak:
                        forest[neig[0],neig[1]] = AB
                    continue
                    
                elif forest[neig[0],neig[1]] == a:
                    if np.random.random() < m_outbreak:
                        forest[neig[0],neig[1]] = aB
                        
                        immune.remove([neig[0],neig[1]])
                        sicks.append([neig[0],neig[1]])
                    continue
                    
                elif forest[neig[0],neig[1]] == S:
                    if np.random.random() < outbreak:
                        forest[neig[0],neig[1]] = B
                        
                        sicks.append([neig[0],neig[1]])
                    continue    
    health()
    
    ### lightning in the forest by the rate of "f"
    if np.random.random() < f:
        susceptibles = sites.copy()
        for s in sicks:
            susceptibles.remove(s)
        for i in immune:
            susceptibles.remove(i)
        lightning(susceptibles)
        
        

## Initialize parameters

In [20]:
outbreak, m_outbreak, h = 0.28, 0.95, 0.68
r, f = 0.06,  0.005

length, width = 100, 100

sicks = []
immune = []
sites = make_site_list(length,width)

forest = np.zeros((length,width))
lightning(sites)


In [22]:
%matplotlib

fig = plt.figure(figsize=(12, 8),facecolor='gray');

ims = [];

for _ in tqdm(range(10)):
    evolve(sites);
    im = plt.imshow(forest, animated=True,cmap=cmap,norm=norm, interpolation='none');
    ims.append([im]);

ani = animation.ArtistAnimation(fig, ims, interval=50, blit=True);

plt.axis('off')

# ani.save('SIRS,%i.mp4'%length);

plt.show();

Using matplotlib backend: Qt5Agg


100%|██████████████████████████████████████████████████████████████████████████████████| 10/10 [00:00<00:00, 32.34it/s]


###### We have two Cooperative Diseases in the lattice, so we can have 9 states for each site in the lattice; S: white, A: red, B: yellow,           AB: orange, a: cyan, b: green, Ab: blue, aB: magenta, ab: black

###### A, B are our diseases and (A, B, AB, Ab, aB) are sick individuals, also (a, b, ab) are immune individuals

<video controls src="1.mp4" />